In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.0f' % x)
import os
import plotly.express as px
from datetime import date

In [ ]:
# df = pd.read_excel('/Users/kai/Desktop/Weekly report_v0.1.xlsx', None) # Mac用
df = pd.read_excel(r'C:\Users\kaihsu\Desktop\業績總表\Weekly report_v0.1.xlsx', None)
df = df['出貨明細']

In [ ]:
today = date.today()
cur_month = today.strftime("%B")
cur_year = int(today.strftime('%Y'))


In [ ]:
df['數量'] = df['數量'].astype('int')
df['本國幣別NTD'] = df['本國幣別NTD'].astype('int')

## 表格

In [ ]:
table = df.query('BG == "WIP" and 預交年份 == "2020" and 預交月份 == "November" and 狀態 != "待通知"').groupby('Group')['本國幣別NTD', '數量'].sum().reset_index()

In [ ]:
table.columns = ['客戶名稱', '營業額', '出貨數量']
format_dict = {'營業額': '{0:,.0f}', '出貨數量': '{0:,.0f}'}
table = table.style.format(format_dict).highlight_max(color='lightgreen').highlight_min(color='red').set_caption('WIP月銷售統計').hide_index()


In [ ]:
# test = test.groupby([pd.Grouper('Group'), pd.Grouper(key='預交日期', freq='Q')])['本國幣別NTD'].sum().reset_index().style.format({'本國幣別NTD':'{0:,.0f}'})

## Treemap

In [ ]:
treemap = df[(df['預交年份'] == cur_year) & (df['BG'] == 'WIP') & (df['狀態'].str.contains('出'))]

In [ ]:
treemap = treemap[treemap['預交月份'] == cur_month]
tree_map = treemap.groupby(['BG', '負責業務', 'Group', '品名'])[['本國幣別NTD', '數量']].sum().reset_index()

In [ ]:
fig = px.treemap(tree_map, path=['BG', '負責業務', 'Group', '品名'], values='本國幣別NTD', color='數量',
                title='Treemap',
                labels={'BG': 'WIP',
                       '負責業務': '負責業務',
                       'Group': '客戶名稱',
                       '品名': '品名'})
fig.show()

In [ ]:
# month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
#                'August', 'September', 'October', 'November', 'December']
# df['預交月份'] = pd.Categorical(df['預交月份'], ordered=True, categories=month_order)
# fig1 = px.histogram(df, x='預交月份', y='本國幣別NTD', barmode='group', color='Group', category_orders={'預交月份': month_order},
#                    labels={'預交月份':'月份', '本國幣別NTD': '營業額', 'Group': '客戶名稱'})
# fig1.show()

## YoY每月出貨圖

In [ ]:
yoy_plot = df[(df['BG'] == 'WIP') & (df['預交年份'] == cur_year) & (df['狀態'] != '待通知')]

In [ ]:

yoy_plot = yoy_plot.groupby(['預交年份', 'Group', '預交月份'])[['數量', '本國幣別NTD']].sum().reset_index()

In [ ]:
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
               'August', 'September', 'October', 'November', 'December']
yoy_plot['預交月份'] = pd.Categorical(yoy_plot['預交月份'], ordered=True, categories=month_order)
yoy_plot = yoy_plot.sort_values(by='預交月份')

In [ ]:
yoy_plot['預交年份'] = yoy_plot['預交年份'].astype(str)
yoy_fig = px.bar(yoy_plot, x='預交月份', y='本國幣別NTD', color='Group', category_orders={'預交月份':month_order}, 
                 title='各月營業額', 
                labels={'Group': '客戶名稱',
                       '本國幣別NTD': '金額',
                       '預交月份': '月份'})
yoy_fig.show()

In [ ]:
with open('WIP月營收.html', 'a') as f:
    f.write(table.render()) # Styler之後要用Render
    f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(yoy_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.close()